In [1]:
import cv2 as cv
import numpy as np

In [2]:
# 1. read in the class names and assign random colors to each
with open('../classes.txt', 'r') as f:
    class_names = f.read().strip().split('\n')

colors = np.random.uniform(0, 255, size=(len(class_names), 3))
print(class_names)
print(colors)

['window', 'window-outside']
[[143.12041442  55.22259916 200.62702689]
 [ 22.92626708 187.90448728 175.48843884]]


In [3]:
# 2. load the model
model = cv.dnn.readNetFromONNX('../best.onnx')

In [4]:
# 3. read in the image and prepare it for model input
image = cv.imread('../window_example.jpg')
height, width, _ = image.shape

blob = cv.dnn.blobFromImage(
    image=image,
    size=(640, 640),
    mean=(104, 117, 123),
    #swapRB=True,  # openCV reads BGR color, so this swaps to RGB to match the model
)
model.setInput(blob)
output = model.forward()

In [5]:
output[0, :, :].T.shape

(8400, 6)

In [10]:
# Note, the shape is (1, 6, 8400), so the first two axes do nothing
for detection in output[0, :, :].T:
    bbox_x, bbox_y, bbox_width, bbox_height = detection[:4]
    probs = detection[4:]
    class_id = np.argmax(probs)
    max_prob = probs[class_id]
    if max_prob > 0.1:
        print(bbox_x, bbox_y, bbox_width, bbox_height)
        class_name = class_names[class_id]
        color = colors[class_id]
        cv.rectangle(image, (bbox_x, bbox_y), (bbox_width, bbox_height), (0, 255, 0), 2)
        cv.putText(image, class_name, (bbox_x, bbox_y - 5), cv.FONT_HERSHEY_SIMPLEX, 1, color, 2)

cv.imshow('image', image)
cv.imwrite('image_result.jpg', image)
cv.waitKey()
cv.destroyAllWindows()


352.05365 161.22191 109.94003 70.03549


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2
